In [ ]:
import pandas as pd
import pennylane as qml
import numpy as np
import matplotlib.pyplot as plt
import math

### Lectura del fichero CSV

In [ ]:
data = pd.read_csv('Datos_Montabilidad_Anom_447-6-1.csv',delimiter=';')

# Filtrado y construcción del dataframe binario
df_binario = data.iloc[:, 2:-2].apply(lambda x: x > 0).astype(int)

#Separar por vectores train y test
train_data = df_binario[data['Set'] == 'Train']
test_data = df_binario[data['Set'] == 'Test']

### Implementación de SWAP Test (2 vectores N columnas)
***Cuando subamos la cantidad de vectores hay que replantear cuantos wires (qbits) utilizar.*

Se calculan los qubits necesarios a partir de la dimensionalidad del vector (cantidad de columnas). El vector se define como:

$\mathbf{x} = (x_1, x_2, x_3, \ldots, x_n)$


Los qubits que vamos a necesitar para codificar (AmplitudeEmbedding) son $ qubits = dim(\mathbf{x})$, ademas:
* Para cada dimension necesitamos un qbits ancilla
* para cada vector necesitamos un qubit por dimentsion

Si tenemos N vectores necesitaremos multiplicar N+1 veces los qubits requeridos para codificar N por cada vector y +1 para los qubits ancilla.



In [ ]:
N = 128  # Numero de componentes
train_vectors = train_data.iloc[:10, :].to_numpy()
test_vectors = test_data.iloc[:2, :].to_numpy()
#train_vector = np.zeros((1,128))
#train_vector[0, -1] = 1
#test_vector =  np.ones((1,128))
#test_vector[0, -1] = 0
#print(train_vector[:,:2])
#print(test_vector)
#tensor_product = np.kron(test_vector, train_vector)
#print("\n")
#print(tensor_product)

In [ ]:
#Amplitude Embedding
qubit_number_AmplitudeEmbedding = int(np.ceil(np.log2(N))) if np.log2(N) > 0 else 1
qubits_totales_amplitude = 3*qubit_number_AmplitudeEmbedding

dev = qml.device("default.qubit", qubits_totales_amplitude)
#dev_qiskit = qml.device("qiskit.aer", qubits_totales_amplitude , backend='aer_simulator_statevector')

@qml.qnode(dev)
@qml.transforms.merge_amplitude_embedding
def circuit(test_vector, train_vector):
    qml.AmplitudeEmbedding(features=test_vector, wires=range(qubit_number_AmplitudeEmbedding,2*qubit_number_AmplitudeEmbedding), normalize=True, pad_with=0.)
    qml.AmplitudeEmbedding(features=train_vector, wires=range(2*qubit_number_AmplitudeEmbedding,3*qubit_number_AmplitudeEmbedding), normalize=True, pad_with=0.)
    ancillea = []
    for i in range(qubit_number_AmplitudeEmbedding):
        qml.Hadamard(i)
        anc = i
        ancillea.append(anc)
        first_state = i+qubit_number_AmplitudeEmbedding
        second_state = i+ 2*qubit_number_AmplitudeEmbedding
        qml.CSWAP(wires=[anc, first_state, second_state])
        qml.Hadamard(i)
    return qml.expval(qml.operation.Tensor(*[qml.PauliZ(i) for i in ancillea]))
    #qml.probs(wires = range(len(ancillea)))
#print(qml.draw(circuit)(test_vector, train_vector))
#print (circuit(test_vector, train_vector))

In [13]:
#estado_0 = circuit(test_vector, train_vector)
iteraciones = range(int(np.ceil(len(test_data.iloc[0,:])/N)))
outliers = []
for w in range(len(test_vectors)):
    for j in range(len(train_vectors)):
        print("\nComparacion -> (train: " + str(j),"test: " + str(w)+")")
        estados  = []
        for i in iteraciones:
            test_vector_particion = test_vectors[w:(w+1),(N*i):((i+1)*N)]
            train_vector_particion = train_vectors[j:(j+1),(N*i):((i+1)*N)]
            
            estado = circuit(test_vector_particion, train_vector_particion)
            estados.append(estado)
            print(estado)

            #print(test_vector[:,(N*i):((i+1)*N)])
            #print(train_vector[:,(N*i):((i+1)*N)])
        estado_total = round(np.sum(estados),2)
        print("Expval total: " + str(estado_total))
        if not estado_total > 0 :
            outliers.append(test_vectors[j:(j+1),:])


Comparacion -> (train: 0 test: 0)
[0.12564103]
[0.2109375]
[0.03174603]
[0.04736842]
[0.05797101]
[0.20915033]
[0.33333333]
Expval total: 1.02

Comparacion -> (train: 1 test: 0)
[0.2967033]
[0.29347826]
[0.13186813]
[0.25]
[0.23148148]
[0.22222222]
[0.66666667]
Expval total: 2.09

Comparacion -> (train: 2 test: 0)
[0.19838057]
[0.39705882]
[0.44444444]
[0.20833333]
[0.03947368]
[0.30864198]
[0.66666667]
Expval total: 2.26

Comparacion -> (train: 3 test: 0)
[0.17582418]
[0.14583333]
[0.32183908]
[0.21176471]
[0.16333333]
[0.17777778]
[0.44444444]
Expval total: 1.64

Comparacion -> (train: 4 test: 0)
[0.13461538]
[0.24107143]
[0.28741497]
[0.2]
[0.12]
[0.23684211]
[0.33333333]
Expval total: 1.55

Comparacion -> (train: 5 test: 0)
[0.17582418]
[0.2688172]
[0.32183908]
[0.08]
[0.16333333]
[0.225]
[0.66666667]
Expval total: 1.9

Comparacion -> (train: 6 test: 0)
[0.13186813]
[0.3375]
[0.40238095]
[0.11428571]
[0.03947368]
[0.30864198]
[0.33333333]
Expval total: 1.67

Comparacion -> (train: